Goal: Predicting the outcome of games


Step 1: Finding key variables

Dean Oliver's - FG%, TO, OR, FTA

the player should:

Shoot a high field goal percentage or help teammates shoot a high percentage through assists and playmaking.
Avoid turnovers.
Get offensive rebounds.
Draw fouls.
At the defensive end, the player should:

Make offensive players take low percentage shots.
Cause turnovers.
Box out and prevent offensive rebounds.
Avoid fouls.

Step 2: Create a model that helps us predict a game.

Create a metric that gives us how a player impacts winning based on Dean Oliver.

Given pregame lineups, determine who will win a game.

In [1]:
import pandas as pd
import statsmodels.api as sm
import numpy as np
import matplotlib.pyplot as plt
from statistics import median

%matplotlib inline

In [2]:
from datetime import datetime, timedelta, date
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import BernoulliNB

In [3]:
teamBoxScore = pd.read_csv("nba-enhanced-stats/2012-18_teamBoxScore.csv")
teamResult = teamBoxScore['teamRslt']
teamBoxScore.head()

,gmDate,gmTime,seasTyp,offLNm1,offFNm1,offLNm2,offFNm2,offLNm3,offFNm3,teamAbbr,...,opptFIC40,opptOrtg,opptDrtg,opptEDiff,opptPlay%,opptAR,opptAST/TO,opptSTL/TO,poss,pace
0,2012-10-30,19:00,Regular,Brothers,Tony,Smith,Michael,Workman,Haywoode,WAS,...,61.6667,105.6882,94.4447,11.2435,0.4390,16.7072,1.0476,33.3333,88.9409,88.9409
1,2012-10-30,19:00,Regular,Brothers,Tony,Smith,Michael,Workman,Haywoode,CLE,...,56.0417,94.4447,105.6882,-11.2435,0.3765,18.8679,2.0000,84.6154,88.9409,88.9409
2,2012-10-30,20:00,Regular,McCutchen,Monty,Wright,Sean,Fitzgerald,Kane,BOS,...,80.8333,126.3381,112.6515,13.6866,0.5244,19.8287,3.1250,100.0000,94.9832,94.9832
3,2012-10-30,20:00,Regular,McCutchen,Monty,Wright,Sean,Fitzgerald,Kane,MIA,...,62.7083,112.6515,126.3381,-13.6866,0.4643,18.8501,1.5000,25.0000,94.9832,94.9832
4,2012-10-30,22:30,Regular,Foster,Scott,Zielinski,Gary,Dalen,Eric,DAL,...,58.6458,99.3678,108.1034,-8.7356,0.5000,18.6567,1.7143,42.8571,91.5790,91.5790


In [4]:
playerBoxScore = pd.read_csv("nba-enhanced-stats/2012-18_playerBoxScore.csv")
playerBoxScore.head()

,gmDate,gmTime,seasTyp,playLNm,playFNm,teamAbbr,teamConf,teamDiv,teamLoc,teamRslt,...,playFT%,playORB,playDRB,playTRB,opptAbbr,opptConf,opptDiv,opptLoc,opptRslt,opptDayOff
0,2012-10-30,19:00,Regular,Price,A.J.,WAS,East,Southeast,Away,Loss,...,1.0,1,1,2,CLE,East,Central,Home,Win,0
1,2012-10-30,19:00,Regular,Ariza,Trevor,WAS,East,Southeast,Away,Loss,...,0.5,1,2,3,CLE,East,Central,Home,Win,0
2,2012-10-30,19:00,Regular,Okafor,Emeka,WAS,East,Southeast,Away,Loss,...,0.5,5,2,7,CLE,East,Central,Home,Win,0
3,2012-10-30,19:00,Regular,Beal,Bradley,WAS,East,Southeast,Away,Loss,...,1.0,0,3,3,CLE,East,Central,Home,Win,0
4,2012-10-30,19:00,Regular,Booker,Trevor,WAS,East,Southeast,Away,Loss,...,0.0,1,0,1,CLE,East,Central,Home,Win,0


In [5]:
standings = pd.read_csv("nba-enhanced-stats/2012-18_standings.csv")
standings.head()

,stDate,teamAbbr,rank,rankOrd,gameWon,gameLost,stk,stkType,stkTot,gameBack,...,rel%Indx,mov,srs,pw%,pyth%13.91,wpyth13.91,lpyth13.91,pyth%16.5,wpyth16.5,lpyth16.5
0,2012-10-30,ATL,3,3rd,0,0,-,-,0,0.5,...,0.0,0.0,0.0,0.500,0.0000,0.0000,82.0000,0.000,0.000,82.000
1,2012-10-30,BKN,3,3rd,0,0,-,-,0,0.5,...,0.0,0.0,0.0,0.500,0.0000,0.0000,82.0000,0.000,0.000,82.000
2,2012-10-30,BOS,14,14th,0,1,L1,loss,1,1.0,...,0.0,-13.0,-13.0,0.072,0.1687,13.8334,68.1666,0.131,10.742,71.258
3,2012-10-30,CHA,3,3rd,0,0,-,-,0,0.5,...,0.0,0.0,0.0,0.500,0.0000,0.0000,82.0000,0.000,0.000,82.000
4,2012-10-30,CHI,3,3rd,0,0,-,-,0,0.5,...,0.0,0.0,0.0,0.500,0.0000,0.0000,82.0000,0.000,0.000,82.000


In [6]:
# Set some variables to hold test, dev, and training data.
test_data, test_labels = teamBoxScore[12500:], teamResult[12500:]
dev_data, dev_labels = teamBoxScore[11000:12500], teamResult[11000:12500]
train_data, train_labels = teamBoxScore[:11000], teamResult[:11000]
mini_train_data, mini_train_labels = teamBoxScore[:1000], teamResult[:1000]

In [7]:
def x_days_before(s, x):
    date = datetime.strptime(s, "%Y-%m-%d")
    modified_date = date - timedelta(days=x)
    return modified_date

In [8]:
#Naive Model - predicts team with better record.. if teams are tied, then predicts home team

#for every game in dev_data:
#  if there exists day_before record of both teams:
#    set record1, record2 to be the day_before_record of teams
#  if record1 != record2:
#    predict better record
#  else:
#    predict home team


temp_standings = standings[['stDate', 'teamAbbr', 'gameWon', 'gameLost']]
correct = 0
wrong = 0

for row in teamBoxScore.iterrows():
    date = row[1]['gmDate'] #30
    day_before = x_days_before(date, 1) #29
    first_date = datetime.strptime(temp_standings.iloc[0]['stDate'], "%Y-%m-%d") #30
    
    if day_before > first_date:
        for i in range(30):
            temp_standings = temp_standings.drop(temp_standings.index[0])
    
    team1 = row[1]['teamAbbr']
    team2 = row[1]['opptAbbr']

    if day_before == first_date:
        #FIX THIS
        dayb4 = datetime.strftime(day_before, "%Y-%m-%d")
        dayb4_df = temp_standings.loc[temp_standings['stDate'] == dayb4]
        
        team1Rec = dayb4_df.loc[dayb4_df['teamAbbr'] == team1]
        team2Rec = dayb4_df.loc[dayb4_df['teamAbbr'] == team2]
        
        team1Over500 = team1Rec.iloc[0]['gameWon'] - team1Rec.iloc[0]['gameLost']
        team2Over500 = team2Rec.iloc[0]['gameWon'] - team2Rec.iloc[0]['gameLost']
        
        if team1Over500 > team2Over500:
            prediction = 'Win'
        elif team1Over500 < team2Over500:
            prediction = 'Loss'
        elif row[1]['teamLoc'] == 'Home':
            prediction = 'Win'
        else:
            prediction = 'Loss'
    else:
        if row[1]['teamLoc'] == 'Home':
            prediction = 'Win'
        else:
            prediction = 'Loss'
    if prediction == row[1]['teamRslt']:
        correct += 1
    else:
        wrong += 1

print(correct)
print(wrong)

9549
5209


Naive model has an accuracy of ~64.7%. We should aim for our model to get above this mark.

In [9]:
#Fill a list with names of officials.
off_set = set()

for row in teamBoxScore.iterrows():
    off1 = str(row[1]['offFNm1'] + ' ' + row[1]['offLNm1'])
    off_set.add(off1)
    
    off2 = str(row[1]['offFNm2']) + ' ' + str(row[1]['offLNm2'])
    off_set.add(off2)
    
    off3 = str(row[1]['offFNm3']) + ' ' + str(row[1]['offLNm3'])
    off_set.add(off3)
    
off_list = list(sorted(off_set))
off_len = len(off_list)

In [10]:
# Create a matrix of which officials officiated which games
# Each game indicates a unique game
# 1 indicates a game an official worked in
off_matrix = np.empty((int(len(teamBoxScore.index) / 2), off_len))

count = -1
for row in teamBoxScore.iterrows():
    count += 1
    
    #managing repeating games
    if count % 2 == 1:
        continue

    off1 = str(row[1]['offFNm1'] + ' ' + row[1]['offLNm1'])
    off1_index = off_list.index(off1)

    off2 = str(row[1]['offFNm2']) + ' ' + str(row[1]['offLNm2'])
    off2_index = off_list.index(off2)
    
    off3 = str(row[1]['offFNm3']) + ' ' + str(row[1]['offLNm3'])
    off3_index = off_list.index(off3)
    
    off_matrix[count // 2][off1_index] = 1
    off_matrix[count // 2][off2_index] = 1
    off_matrix[count // 2][off3_index] = 1
    
off_df = pd.DataFrame(off_matrix, columns = off_list)
off_df.head()

,Aaron Smith,Ben Taylor,Bennett Salvatore,Bennie Adams,Bill Kennedy,Bill Spooner,Brandon Adair,Brenda Pantoja,Brent Barnaky,Brett Nansel,...,Steven Anderson,Tom Washington,Tony Brothers,Tony Brown,Tre Maddox,Tyler Ford,Violet Palmer,Vladimir Voyard-Tadal,Zach Zarba,nan nan
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [11]:
#Fill a list with names of players.
player_set = set()

for row in playerBoxScore.iterrows():
    player = str(row[1]['playFNm'] + ' ' + row[1]['playLNm'])
    player_set.add(player)
    
temp_list = list(sorted(player_set))

player_list = []
for player in temp_list:
    player_list.append(player)
    player_list.append(player + " - Away")
    
player_len = len(player_list)

In [12]:
# Create a matrix of which players played in which games
# 1 indicates a player playing in the game

player_matrix = np.empty((int(len(teamBoxScore.index) / 2), player_len))

count = -1
current_team = 'wow I really enjoyed this course Amit'
for row in playerBoxScore.iterrows():
    if row[1]['teamAbbr'] != current_team:
        current_team = row[1]['teamAbbr']
        count += 1
    player = str(row[1]['playFNm'] + ' ' + row[1]['playLNm'])
    player_index = player_list.index(player)
    
    if row[1]['teamLoc'] == 'Home':
        player_matrix[count // 2][player_index] = 1
    else:
        player_matrix[count // 2][player_index + 1] = 1
    
player_df = pd.DataFrame(player_matrix, columns = player_list)
player_df.head()

,A.J. Price,A.J. Price - Away,Aaron Brooks,Aaron Brooks - Away,Aaron Gordon,Aaron Gordon - Away,Aaron Gray,Aaron Gray - Away,Aaron Hammons,Aaron Hammons - Away,...,Zach Randolph,Zach Randolph - Away,Zachary LaVine,Zachary LaVine - Away,Zaza Pachulia,Zaza Pachulia - Away,Zhou Qi,Zhou Qi - Away,Zoran Dragić,Zoran Dragić - Away
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [13]:
# Find EFG% for each team in the season leading up to a game
# Do this by tracking their FGM, 3PM, and FGA

#Creating a list of names
team_names_set = set()
for row in teamBoxScore.iterrows():
    team_names_set.add(row[1]['teamAbbr'])
    team_names_set.add(row[1]['opptAbbr'])
    
    if len(team_names_set) == 30:
        break
team_names_list = list(sorted(team_names_set))

#matrix keeping track of efg before every game
efg_matrix = np.zeros((len(teamBoxScore.index) // 2, 10))
efg_columns_list = ['teamEFG%', 'opptEFG%','teamEFG%A', 'opptEFG%A','teamPF', 'opptPF',
                    'teamTO', 'opptTO', 'teamOREB', 'opptOREB']

#matrix calculating each team's cumulative EFG%
team_stats_matrix = np.zeros((30, 11)) ###(ADJUSTABLE)

prev_date = datetime.strptime('2012-10-29', "%Y-%m-%d")
count = -1
for row in teamBoxScore.iterrows():
    count += 1
    
    #managing repititons
    if row[1]['teamLoc'] == 'Away':
        continue
    
    #finding index of the two teams playing
    team = row[1]['teamAbbr']
    oppt = row[1]['opptAbbr']
    team_index = team_names_list.index(team)
    oppt_index = team_names_list.index(oppt)
    
    #helper function that accesses the team_stats_matrix
    team_row = team_stats_matrix[team_index]
    oppt_row = team_stats_matrix[oppt_index]
    
    #updating efg_matrix
    efg_matrix[count // 2][0] = team_row[3]
    efg_matrix[count // 2][2] = team_row[7]
    efg_matrix[count // 2][4] = team_row[8]
    efg_matrix[count // 2][6] = team_row[9]
    efg_matrix[count // 2][8] = team_row[10]
    efg_matrix[count // 2][1] = oppt_row[3]
    efg_matrix[count // 2][3] = oppt_row[7]
    efg_matrix[count // 2][5] = oppt_row[8]
    efg_matrix[count // 2][7] = oppt_row[9]
    efg_matrix[count // 2][9] = oppt_row[10]

    #updating team_stats_matrix for this game
    team_row[0] += row[1]['teamFGM']
    team_row[1] += row[1]['team3PM']
    team_row[2] += row[1]['teamFGA']
    team_row[3] = (team_row[0] + 0.5 * (team_row[1])) / team_row[2] #teamEFG%
    team_row[4] += row[1]['opptFGM']
    team_row[5] += row[1]['oppt3PM']
    team_row[6] += row[1]['opptFGA']
    team_row[7] = (team_row[4] + 0.5 * (team_row[5])) / team_row[6] #teamEFG%A
    team_row[8] += row[1]['teamPF'] #teamPF
    team_row[9] += row[1]['teamTO'] #teamTO
    team_row[10] += row[1]['teamORB'] #teamOREB
    
    oppt_row[0] += row[1]['opptFGM']
    oppt_row[1] += row[1]['oppt3PM']
    oppt_row[2] += row[1]['opptFGA']
    oppt_row[3] = (oppt_row[0] + 0.5 * (oppt_row[1])) / oppt_row[2] #opptEFG%
    oppt_row[4] += row[1]['teamFGM']
    oppt_row[5] += row[1]['team3PM']
    oppt_row[6] += row[1]['teamFGA']
    oppt_row[7] = (oppt_row[4] + 0.5 * (oppt_row[5])) / oppt_row[6] #opptEFG%A
    oppt_row[8] += row[1]['opptPF'] #opptPF
    oppt_row[9] += row[1]['opptTO'] #opptTO
    oppt_row[10] += row[1]['opptORB'] #opptORB
    
    current_date = datetime.strptime(row[1]['gmDate'], "%Y-%m-%d")
    if (current_date - prev_date).days > 90:
        team_stats_matrix = np.zeros((30, 11)) ###(ADJUSTABLE)

    prev_date = current_date

column_names = ['FGM', '3PM', 'FGA', 'EFG%', 'FGMA', '3PMA', 'FGAA', 'EFG%A', 'PF', 'TO', 'ORB']
totals_df = pd.DataFrame(team_stats_matrix, index = team_names_list, columns = column_names)
#loading efg_matrix data onto a dataframe

stats_df = pd.DataFrame(efg_matrix, columns = efg_columns_list)
stats_df.tail()

,teamEFG%,opptEFG%,teamEFG%A,opptEFG%A,teamPF,opptPF,teamTO,opptTO,teamOREB,opptOREB
7374,0.512067,0.526666,0.526119,0.522674,1552.0,1721.0,1178.0,1188.0,717.0,804.0
7375,0.534821,0.532306,0.492390,0.527453,1794.0,1735.0,1344.0,1113.0,876.0,680.0
7376,0.527256,0.516550,0.519144,0.514405,1621.0,1713.0,1192.0,1279.0,824.0,866.0
7377,0.511004,0.528627,0.504315,0.506404,1577.0,1589.0,1101.0,1194.0,828.0,725.0
7378,0.502292,0.551899,0.543446,0.521482,1625.0,1573.0,1116.0,1131.0,771.0,731.0


In [14]:
# Get back to back status for each team as well as consecutive away games

#Start with teamBoxScore 
teams = teamBoxScore[["teamAbbr", "teamLoc", "teamRslt", "gmDate"]]

#convert gmDate into date object
def make_date(x):
    return datetime.strptime(x, '%Y-%m-%d').date()

#add datetime column, drop date strings column
gmDate_date = list(map(make_date, teams['gmDate']))
teams['gmDate_date'] = gmDate_date
teams.drop(['gmDate'], axis = 1, inplace = True)

sorted_teams = teams.groupby('teamAbbr')
sorted_teams.first()

# Build a dictionary with keys being the teams
team_names = list(teamBoxScore.teamAbbr.unique())
team_dict = {}
for teams in team_names:
    xx = sorted_teams.get_group(teams)
    xx = xx.reset_index()
    
    for ind, row in xx.iterrows():
        if ind == 0:
            xx.loc[ind, "back_to_back"] = 0
        else:
            if row['gmDate_date'] == (xx.loc[ind-1,'gmDate_date'] + timedelta(days=1)):
                xx.loc[ind, "back_to_back"] = 1
            else:
                xx.loc[ind, "back_to_back"] = 0
    Counter = 0            
    for ind, row in xx.iterrows():
        if row['teamLoc'] == "Away":
            Counter += 1
            xx.loc[ind, "consecutive_away_games"] = Counter
        else:
            Counter = 0
            xx.loc[ind, "consecutive_away_games"] = Counter
    
    team_dict[teams] = xx
    

# Combine the values in dictionary and then sort them back into order
frames = []
for abbr in list(team_dict.keys()):
    frames.append(team_dict[abbr][['index','back_to_back',"consecutive_away_games"]])

result = pd.concat(frames)
result = result.sort_values('index')
result.set_index('index', inplace = True)

#Combining to teamBoxScore
newBox = pd.concat([ teamBoxScore, result[['back_to_back','consecutive_away_games']] ], axis=1, sort=False)

#delete duplicates/repetitions
testBox = newBox
for row in testBox.itertuples():
    if row[13] == 'Away':
        testBox.drop([row[0]], inplace = True)

/Users/jjsahabu/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if sys.path[0] == '':
/Users/jjsahabu/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [15]:
#run once only
testBox.reset_index(inplace=True)
b2b_df = testBox['back_to_back']
away_streak_df = testBox['consecutive_away_games']

In [16]:
team_result_df = pd.DataFrame(testBox['teamRslt'])

In [17]:
win_loss = []
for row in team_result_df.itertuples():
    if row[1] == 'Win':
        win_loss.append(1)
    else:
        win_loss.append(0)

In [18]:
len(win_loss)

7379

In [19]:
#Combining our dataframes
result = pd.concat([off_df, 
                    player_df, 
                    stats_df, 
                    b2b_df, 
                    away_streak_df], axis=1, sort=False)
result.columns

Index(['Aaron Smith', 'Ben Taylor', 'Bennett Salvatore', 'Bennie Adams',
       'Bill Kennedy', 'Bill Spooner', 'Brandon Adair', 'Brenda Pantoja',
       'Brent Barnaky', 'Brett Nansel',
       ...
       'teamEFG%A', 'opptEFG%A', 'teamPF', 'opptPF', 'teamTO', 'opptTO',
       'teamOREB', 'opptOREB', 'back_to_back', 'consecutive_away_games'],
      dtype='object', length=1981)

In [20]:
# General libraries.
import re

# SK-learn libraries for learning.
from sklearn.pipeline import Pipeline
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import MultinomialNB

# SK-learn libraries for evaluation.
from sklearn.metrics import confusion_matrix
from sklearn import metrics
from sklearn.metrics import classification_report

# SK-learn libraries for feature extraction from text.
from sklearn.feature_extraction.text import *

import nltk

In [21]:
# Set some variables to hold test, dev, and training data.
test_data, test_labels = result[7000:], win_loss[7000:]
dev_data, dev_labels = result[6500:7000], win_loss[6500:7000]
train_data, train_labels = result[:6500], win_loss[:6500]
mini_train_data, mini_train_labels = result[:500], win_loss[:500]

In [22]:
# Set up log regression
LR = LogisticRegression()
LR.fit(train_data, train_labels)
pred_labels_LR = LR.predict(dev_data)
print("The accuracy of Log Regression model is: " + 
      str(LR.score(dev_data, dev_labels)))


#NaiveBayes
# model = BernoulliNB(binarize = 0.5)
# model.fit(mini_train_data, mini_train_labels)
# predicted_labels = model.predict(dev_data)
# print("The accuracy of Bernoulli's Naive Bayes Model is: " + 
#       str(model.score(dev_data, dev_labels)))

/Users/jjsahabu/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


The accuracy of Log Regression model is: 0.602


In [23]:
deanBoxScore = teamBoxScore[['teamFG%', 'teamTO', 'teamORB', 'teamPF'
                             , 'opptFG%', 'opptTO', 'opptORB', 'opptPF']]
deanBoxScore.head()

deanResult = teamBoxScore['teamRslt']

deanResult.head()

0    Loss
1     Win
2    Loss
3     Win
4     Win
Name: teamRslt, dtype: object

In [24]:
# Set some variables to hold test, dev, and training data.
test_data1, test_labels1 = deanBoxScore[12500:], deanResult[12500:]
dev_data1, dev_labels1 = deanBoxScore[11000:12500], deanResult[11000:12500]
train_data1, train_labels1 = deanBoxScore[:11000], deanResult[:11000]
mini_train_data1, mini_train_labels1 = deanBoxScore[:1000], deanResult[:1000]

In [25]:
#KNN
for k in range(1, 20):
    model = KNeighborsClassifier(n_neighbors = k)
    model.fit(mini_train_data1, mini_train_labels1)
    print("The accuracy of KNN with " + str(k) + " neighbors is " + str(model.score(dev_data1, dev_labels1)))


The accuracy of KNN with 1 neighbors is 0.5453333333333333
The accuracy of KNN with 2 neighbors is 0.55
The accuracy of KNN with 3 neighbors is 0.5586666666666666
The accuracy of KNN with 4 neighbors is 0.5626666666666666
The accuracy of KNN with 5 neighbors is 0.5786666666666667
The accuracy of KNN with 6 neighbors is 0.5793333333333334
The accuracy of KNN with 7 neighbors is 0.5973333333333334
The accuracy of KNN with 8 neighbors is 0.584
The accuracy of KNN with 9 neighbors is 0.5786666666666667
The accuracy of KNN with 10 neighbors is 0.5713333333333334
The accuracy of KNN with 11 neighbors is 0.5786666666666667
The accuracy of KNN with 12 neighbors is 0.588
The accuracy of KNN with 13 neighbors is 0.5933333333333334
The accuracy of KNN with 14 neighbors is 0.594
The accuracy of KNN with 15 neighbors is 0.5973333333333334
The accuracy of KNN with 16 neighbors is 0.5926666666666667
The accuracy of KNN with 17 neighbors is 0.5946666666666667
The accuracy of KNN with 18 neighbors is 0

In [26]:
#NaiveBayes
model = BernoulliNB(binarize = 0.5)
model.fit(mini_train_data1, mini_train_labels1)
predicted_labels = model.predict(dev_data1)
print("The accuracy of Bernoulli's Naive Bayes Model is: " + 
      str(model.score(dev_data1, dev_labels1)))

The accuracy of Bernoulli's Naive Bayes Model is: 0.6313333333333333
